# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d18/93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Shanghai, Shanghai, China**, and the stations the data comes from are shown on the map below.

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('BinSize_d{}.csv'.format(binsize))
    
    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()


leaflet_plot_stations(18,'93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628')

In [ ]:
df1 = pd.read_csv('93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628.csv')
df2 = pd.read_csv('BinSize_d18.csv')
df1.head()


In [ ]:
df2.head()


In [ ]:
df3 = df2[df2['hash'] == '93c648398ff85fad51308f4ff8d11c2e8d8e66392462ffe79f3fb628']
df3.head()

In [ ]:
df_total = pd.merge(left=df1,right=df2,how="inner", left_on='ID', right_on="ID")
df_total

In [ ]:
df_total = df_total[['ID','Date','Element', 'Data_Value','NAME']]
df_total

In [ ]:
df_total = df_total[df_total.loc[:,'ID'].str.contains(r'^CHM') & \
                    (df_total.loc[:, 'NAME'] == 'SHANGHAI')]
df_total = df_total.copy()
df_total.loc[:,'Date'] = pd.to_datetime(df_total.loc[:,'Date'])

df_total.loc[:,'Data_Value'] = df_total.loc[:,'Data_Value'].apply(lambda x: x/10)

df_05_14 = df_total[df_total.loc[:,'Date'] < pd.Timestamp('1/1/2015')]
df_05_14 = df_05_14.copy()

df_2015 = df_total[df_total.loc[:,'Date'] > pd.Timestamp('1/1/2015')]
df_2015 = df_2015.copy()





In [ ]:
from datetime import datetime as dt
df_05_14.sort_values(by = 'Date' ,inplace=True)

df_2015.sort_values(by = 'Date' ,inplace=True)


df_05_14['NewDate'] = df_total.Date.dt.strftime("%m-%d")
df_2015['NewDate'] = df_total.Date.dt.strftime("%m-%d")

df_05_14.drop(columns='Date')
df_2015.drop(columns='Date')

df_05_14 = df_05_14[df_05_14['NewDate'] != '02-29']

max_record = df_05_14.groupby('NewDate')['Data_Value'].max().reset_index()
max_record.columns = ['NewDate', 'Data_Value']


min_record = df_05_14.groupby('NewDate')['Data_Value'].min().reset_index()
min_record.columns = ['NewDate', 'Data_Value']


max_record_2015 = df_2015.groupby('NewDate')['Data_Value'].max().reset_index()
max_record_2015.columns = ['NewDate', 'Data_Value']

min_record_2015 = df_2015.groupby('NewDate')['Data_Value'].min().reset_index()
min_record_2015.columns = ['NewDate', 'Data_Value']


max_record_2015_2 = pd.merge(left=max_record_2015, right=max_record, how = 'outer', left_on='NewDate', right_on='NewDate')

min_record_2015_2 = pd.merge(left=min_record_2015, right=min_record, how = 'outer', left_on='NewDate', right_on='NewDate')


In [ ]:
import numpy as np
max_record_2015_2['Data_Value'] = np.where(max_record_2015_2['Data_Value_x'] > max_record_2015_2['Data_Value_y'],\
                                           max_record_2015_2['Data_Value_x'], np.nan)

min_record_2015_2['Data_Value'] = np.where(min_record_2015_2['Data_Value_x'] < min_record_2015_2['Data_Value_y'],\
                                           min_record_2015_2['Data_Value_x'], np.nan)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
dates = pd.date_range("2015-01-01", "2015-12-31" )
df = pd.DataFrame({"Dates" : dates})

max_record_2015_2.sort_values(by='NewDate',inplace=True)
min_record_2015_2.sort_values(by='NewDate',inplace=True)

max_record_2015_2['Dates'] = dates
min_record_2015_2['Dates'] = dates
max_record['Dates'] = dates
min_record['Dates'] = dates

plt.figure(figsize=(20,12))

plt.plot(max_record.loc[:,'Dates'],max_record.loc[:,'Data_Value'],'-')
plt.plot(max_record['Dates'],min_record['Data_Value'],'-')

plt.plot_date(x = max_record_2015_2['Dates'], y = max_record_2015_2['Data_Value'], c = 'red')

plt.plot_date(min_record_2015_2.loc[:,'Dates'],min_record_2015_2.loc[:,'Data_Value'],c = 'blue')


plt.ylabel('Temperature(Celsius $^oC$)', )

plt.legend(['2005-2014 Temp Highest', '2005-2014 Temp Lowest', 'Max Temp 2015', 'Min Temp 2015'])



plt.gca().fill_between(max_record['Dates'].values,max_record['Data_Value'] ,min_record['Data_Value'], facecolor = 'blue', alpha = .25)


import matplotlib.dates as dates
import matplotlib.ticker as ticker



ax = plt.gca()
ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.DayLocator())

monthFmt = dates.DateFormatter('%b')
ax.xaxis.set_major_formatter(monthFmt)

font = {'size'   : 18}

matplotlib.rc('font', **font)

plt.rcParams.update({'font.size': 22})

plt.title('Temperature of Shanghai')

plt.savefig('assignment2.png')

